In [120]:
!pip install pretrainedmodels

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
# !pip install pyyaml==5.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
!python3 -m pip install ../input/nflimpactdetectron/detectron2/detectron2 --no-index --find-links ../input/nflimpactdetectron/wheelz/wheelz

In [121]:
from torch.utils.data import DataLoader, Dataset

In [124]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import cv2
import pickle

In [125]:
from torchvision.ops import box_area, box_iou

In [126]:
import sys
sys.path.append('../input/footballhelmetdetector')
from model import get_predictor
# \, new_model_cfg
from train_helmet_detector import NflImpactTrainer

In [ ]:
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.model_zoo import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader, build_detection_test_loader, \
    DatasetMapper
from detectron2.structures import BoxMode, Instances, Boxes
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.structures import Instances

from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads
from detectron2.modeling import ROI_HEADS_REGISTRY

from typing import Dict, List, Optional, Tuple
import albumentations as A
# from albumentations.pytorch.transforms import ToTensor, ToTensorV2
# from albumentations.augmentations.transforms import Normalize

from albumentations import Compose, RandomCrop, Normalize, HorizontalFlip, Resize
from albumentations.pytorch import ToTensor

In [ ]:
from detectron2.utils.visualizer import Visualizer

In [127]:
from copy import deepcopy 
from tqdm import tqdm

In [ ]:
def convert_video_labels(video_labels: pd.DataFrame):
    
    video_labels = video_labels.rename(columns={'x':'left', 'y':'top', 'w':'width', 'h':'height'})
    
    dataset_dicts = []
    all_videos = video_labels['video'].unique()
    videos_processed = 0
    for video in all_videos:
        frames = video_labels.query("video == @video")
        min_frame = frames['frame'].min()
        max_frame = frames['frame'].max()

        for i in range(min_frame, max_frame + 1):
            player_frames = frames.query("frame == @i")
            vid = video.split('.')[:-1][0]
            img_name = video.split('.')[:-1][0] + '_' + str.format("{}", i).zfill(3) + '.png'
            img_path = os.path.join("../input/nfl-impact-detection-train-frames", vid, img_name)
            if not os.path.exists(img_path):
                continue
            record = {"image_id": img_path, "file_name": img_path, "height": 720, "width": 1280}
            objects = []
            if not player_frames.empty:
                for _, row in player_frames.iterrows():
                    if(row['impact'] == 0):
                        category_id = 0
                    else:
                        category_id = 1
#                     print(f"_ = {_}")
#                     print(f"row = {row}")
                    obj = {
                        "bbox": [row['left'],
                                 row['top'],
                                 row['left'] + row['width'],
                                 row['top'] + row['height']],
                        "bbox_mode": BoxMode.XYXY_ABS,                        
                        "category_id": 0,
                    }
                    objects.append(obj)

            record["annotations"] = objects
            if objects:
                dataset_dicts.append(record)

        print("conversion progress %d/%d" % (videos_processed, len(all_videos)))
        videos_processed += 1
    return dataset_dicts

In [ ]:
@ROI_HEADS_REGISTRY.register()
class Nfl_Crop_ROIHeads(StandardROIHeads):
    def _convert_tl_br(self, w,h,magnitude, x1,y1,x2,y2,):
        assert magnitude >0, 'increase size <0'

        w_box = abs(x2 - x1)*magnitude
        assert w_box >0, 'w_box <0'
        h_box = abs(y2 - y1)*magnitude
        assert h_box >0, 'h_box <0'

        x1 = x1 - w_box if (x1 - w_box) >0 else 0
        y1 = y1 - h_box if (y1 - h_box) >0 else 0

        x2 = x2 + w_box if (x2 + w_box) <w else w
        y2 = y2 + h_box if (y2 + h_box) <h else h

        return torch.tensor([x1,y1,x2,y2]).to('cuda')
#         return torch.tensor([x1,y1,x2,y2])

    
    
    def transform_proposals(self, proposals: List[Instances], magnitude: int):
#         proposals = deepcopy(input_proposals)
#         assert len(proposals) == 1, 'len of proposal != 1'
        for instance in proposals:
            w, h = instance.image_size
            tensor = instance.get('proposal_boxes')
    #         temp = tensor.apply_(lambda x: convert_tl_br(720,1280,magnitude,*x))
            transform_boxes = []
            for i in tensor:    
                transform_boxes.append(self._convert_tl_br(w,h,magnitude,*i))
            transform_boxes = torch.stack(transform_boxes)
            instance.set('proposal_boxes', Boxes(transform_boxes))
        return proposals

    def _forward_box(self, features: Dict[str, torch.Tensor], proposals: List[Instances]):
        """
        Forward logic of the box prediction branch. If `self.train_on_pred_boxes is True`,
            the function puts predicted boxes in the `proposal_boxes` field of `proposals` argument.

        Args:
            features (dict[str, Tensor]): mapping from feature map names to tensor.
                Same as in :meth:`ROIHeads.forward`.
            proposals (list[Instances]): the per-image object proposals with
                their matching ground truth.
                Each has fields "proposal_boxes", and "objectness_logits",
                "gt_classes", "gt_boxes".

        Returns:
            In training, a dict of losses.
            In inference, a list of `Instances`, the predicted instances.
        """
        features = [features[f] for f in self.box_in_features]
        #          
        proposals = self.transform_proposals(proposals, 1)
        box_features = self.box_pooler(features, [x.proposal_boxes for x in proposals])
        box_features = self.box_head(box_features)
        predictions = self.box_predictor(box_features)
        del box_features

        if self.training:
            losses = self.box_predictor.losses(predictions, proposals)
            # proposals is modified in-place below, so losses must be computed first.
            if self.train_on_pred_boxes:
                with torch.no_grad():
                    pred_boxes = self.box_predictor.predict_boxes_for_gt_classes(
                        predictions, proposals
                    )
                    for proposals_per_image, pred_boxes_per_image in zip(proposals, pred_boxes):
                        proposals_per_image.proposal_boxes = Boxes(pred_boxes_per_image)
            return losses
        else:
            pred_instances, _ = self.box_predictor.inference(predictions, proposals)
            return pred_instances

In [ ]:
!mkdir output

In [ ]:
def new_model_cfg(custom_ROIHeads=None):
    cfg = get_cfg()
    model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(model))
#     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    cfg.MODEL.WEIGHTS = "../input/nflimpactdetectron/helmet_detector_model_0093999.pth"
    cfg.MODEL.MASK_ON = False
    cfg.INPUT.RANDOM_FLIP = "none"
    cfg.OUTPUT_DIR = "output"

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.CHECKPOINT_PERIOD = 2000
    cfg.SOLVER.STEPS = (21000, 50000)
    cfg.SOLVER.MAX_ITER = 1000
    cfg.SOLVER.BASE_LR = 0.001
    cfg.TEST.EVAL_PERIOD = 2000
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
    if(custom_ROIHeads):
        cfg.MODEL.ROI_HEADS.NAME = custom_ROIHeads
    

        
    return cfg

# Prepare train and test

In [ ]:
# path = '../input/nfl-csv/Full_4frame_impact.csv'

# video_labels = pd.read_csv(path, index_col=0)
# video_labels = video_labels[video_labels['impact']>0]

# train_gameKey, test_gameKey = train_test_split(video_labels['gameKey'].unique(), test_size= 0.2)

# def check_split(x):
#     if x['gameKey'] in test_gameKey:
#         return 'Test' 
#     else: return 'Train'

# video_labels['split'] = video_labels.apply(lambda x: check_sp lit(x), axis=1)

# train_df = video_labels[(video_labels['split'] == 'Train') & (video_labels['impact'] == 2)]
# test_df = video_labels[(video_labels['split'] == 'Test') & (video_labels['impact'] == 2)]
# len(train_df.groupby('image_name'))
# len(test_df.groupby('image_name'))
# train = convert_video_labels(train_df)
# test = convert_video_labels(test_df)
# with open('e2e_impact2.pickle', 'wb') as f:
#     pickle.dump((train, test), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
!ls 

In [234]:
train_df = pd.read_csv('../input/utils-to-prepare-different-type-of-dataset/well_train_dataset.csv')
test_df = pd.read_csv('../input/utils-to-prepare-different-type-of-dataset/well_test_dataset.csv')

In [231]:
t_df = pd.read_csv('../input/nfl-impact-detection/train_labels.csv').fillna(0)

In [241]:
temp = t_df.apply(lambda x: x['gameKey'] in test_df.gameKey.unique(),axis=1) 
for i in df.groupby('gameKey')

KeyboardInterrupt: 

In [233]:
pd.merge(test_df, t_df)

,gameKey,playID,view,video,frame,label,impactType,confidence,visibility,image_name,x,y,w,h,left,width,top,height,impact
0,57583,82,Endzone,57583_000082_Endzone.mp4,38,H99,Helmet,2.0,2.0,57583_000082_Endzone_038.png,582,311,21,27,582,21,311,27,1.0
1,57583,82,Endzone,57583_000082_Endzone.mp4,38,V68,Helmet,2.0,2.0,57583_000082_Endzone_038.png,562,313,18,15,562,18,313,15,1.0
2,57583,82,Endzone,57583_000082_Endzone.mp4,49,H97,Helmet,3.0,3.0,57583_000082_Endzone_049.png,425,311,15,20,425,15,311,20,1.0
3,57583,82,Endzone,57583_000082_Endzone.mp4,49,V72,Helmet,3.0,3.0,57583_000082_Endzone_049.png,434,316,18,15,434,18,316,15,1.0
4,57583,82,Endzone,57583_000082_Endzone.mp4,58,H90,Helmet,3.0,3.0,57583_000082_Endzone_058.png,238,299,18,31,238,18,299,31,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,58104,352,Sideline,58104_000352_Sideline.mp4,92,H76,Helmet,3.0,2.0,58104_000352_Sideline_092.png,538,282,11,10,538,11,282,10,1.0
287,58104,352,Sideline,58104_000352_Sideline.mp4,92,V92,Helmet,3.0,2.0,58104_000352_Sideline_092.png,536,275,14,10,536,14,275,10,1.0
288,58104,352,Sideline,58104_000352_Sideline.mp4,134,H26,Helmet,2.0,0.0,58104_000352_Sideline_134.png,527,389,13,14,527,13,389,14,1.0
289,58104,352,Sideline,58104_000352_Sideline.mp4,136,V91,Helmet,2.0,2.0,58104_000352_Sideline_136.png,541,398,11,11,541,11,398,11,1.0


In [225]:
pd.merge(test_df,t_df )

,gameKey,playID,view,video,frame,label,impactType,confidence,visibility,image_name,x,y,w,h,left,width,top,height,impact
0,57583,82,Endzone,57583_000082_Endzone.mp4,38,H99,Helmet,2.0,2.0,57583_000082_Endzone_038.png,582,311,21,27,582,21,311,27,1.0
1,57583,82,Endzone,57583_000082_Endzone.mp4,38,V68,Helmet,2.0,2.0,57583_000082_Endzone_038.png,562,313,18,15,562,18,313,15,1.0
2,57583,82,Endzone,57583_000082_Endzone.mp4,49,H97,Helmet,3.0,3.0,57583_000082_Endzone_049.png,425,311,15,20,425,15,311,20,1.0
3,57583,82,Endzone,57583_000082_Endzone.mp4,49,V72,Helmet,3.0,3.0,57583_000082_Endzone_049.png,434,316,18,15,434,18,316,15,1.0
4,57583,82,Endzone,57583_000082_Endzone.mp4,58,H90,Helmet,3.0,3.0,57583_000082_Endzone_058.png,238,299,18,31,238,18,299,31,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,58104,352,Sideline,58104_000352_Sideline.mp4,92,H76,Helmet,3.0,2.0,58104_000352_Sideline_092.png,538,282,11,10,538,11,282,10,1.0
287,58104,352,Sideline,58104_000352_Sideline.mp4,92,V92,Helmet,3.0,2.0,58104_000352_Sideline_092.png,536,275,14,10,536,14,275,10,1.0
288,58104,352,Sideline,58104_000352_Sideline.mp4,134,H26,Helmet,2.0,0.0,58104_000352_Sideline_134.png,527,389,13,14,527,13,389,14,1.0
289,58104,352,Sideline,58104_000352_Sideline.mp4,136,V91,Helmet,2.0,2.0,58104_000352_Sideline_136.png,541,398,11,11,541,11,398,11,1.0


In [ ]:
train = convert_video_labels(train_df)

In [ ]:
test = convert_video_labels(test_df)

In [ ]:
# im = cv2.imread('../input/nfl-impact-detection-train-frames/57583_000082_Endzone/57583_000082_Endzone_001.png')
# dataset_dicts = convert_video_labels(video_labels)
# dataset_dicts = pd.read_pickle('../input/nfl-csv/dataset_dicts.pickle')
# dataset_dicts = pd.read_pickle('../input/nfl-csv/detectron2_4frame_dataset.pickle')

# train, test = train_test_split(dataset_dicts, test_size=0.05, shuffle=False)
# train, test = pd.read_pickle('../input/nfl-csv/e2e_impact2.pickle')

# train = convert_video_labels(df[df['split']=='Train'])
# test = convert_video_labels(df[df['split']=='Test'])


DatasetCatalog.register("nflimpact_train", lambda: train)
DatasetCatalog.register("nflimpact_test", lambda: test)

# classes = ["impact"]
classes = ["helmet"]
# classes = ["helmet", "impact"]
MetadataCatalog.get("nflimpact_train").set(thing_classes=classes)
MetadataCatalog.get("nflimpact_test").set(thing_classes=classes)
# cfg = new_model_cfg('Nfl_Crop_ROIHeads')
cfg = new_model_cfg()
# assert cfg.MODEL.ROI_HEADS.NAME == 'NflROIHeads', 'incorrect ROI_HEADS.NAME'
cfg.DATASETS.TRAIN = ("nflimpact_train",)
cfg.DATASETS.TEST = ("nflimpact_test",)


In [ ]:

print(cfg.MODEL.WEIGHTS)
print(cfg.MODEL.ROI_HEADS.NAME)

In [ ]:
!ls output

In [ ]:
trainer = NflImpactTrainer(cfg)
trainer.resume_or_load(resume=False)
# DetectionCheckpointer(model).load("./output/model_final.pth")
# trainer.train()


In [ ]:
temp = trainer.test(cfg, trainer.model)

In [ ]:
!ls output

In [128]:
def convert_instances(instances_prediction_path):
    instances_predictions = torch.load(instances_prediction_path)
    df = pd.DataFrame(instances_predictions)
    df = df.explode('instances').reset_index()
    df = df.drop(columns=['index'])
    
    output_df = pd.DataFrame(df['instances'].tolist())
    output_df['image_id'] = output_df['image_id'].apply(lambda x: x.split('/')[-1])
    output_df[output_df['score'] > 0.9]
    output_df[['x','y','w','h']] = output_df['bbox'].tolist()
    output_df = output_df.drop(columns = ['bbox'])
    output_df = output_df[output_df['score']>0.98]
    return output_df

In [146]:
df = convert_instances('./output/instances_predictions.pth').fillna(-1)

In [131]:
# img = cv2.imread('../input/nfl-impact-detection/images/57502_000480_Endzone_frame0495.jpg')

In [132]:
# val_loader = build_detection_test_loader(cfg, "nflimpact_test")
# print(inference_on_dataset(trainer.model, val_loader, evaluator))
# # another equivalent way to evaluate the model is to use `trainer.test`

In [164]:
df

,image_id,category_id,score,x,y,w,h
0,57583_000082_Endzone_034.png,0,0.999986,721.682190,219.008087,21.668213,27.973236
1,57583_000082_Endzone_034.png,0,0.999984,963.926819,245.999466,20.739258,28.251358
2,57583_000082_Endzone_034.png,0,0.999980,556.494812,462.678345,21.307373,20.062500
3,57583_000082_Endzone_034.png,0,0.999977,1207.106201,168.313507,22.421387,27.463409
4,57583_000082_Endzone_034.png,0,0.999975,947.314392,189.925537,20.587646,24.750397
...,...,...,...,...,...,...,...
37856,58104_000352_Sideline_187.png,0,0.996708,532.943542,416.640503,12.297852,17.301636
37857,58104_000352_Sideline_187.png,0,0.993401,544.001160,481.328247,12.367310,14.757874
37858,58104_000352_Sideline_187.png,0,0.992077,746.424927,445.458679,12.172363,14.041931
37859,58104_000352_Sideline_187.png,0,0.982033,701.351685,440.789825,12.585571,16.168030


In [192]:
class ImageDatasetBoxes(Dataset) :
    def __init__(self, df=None, transform=None) :
        self.df = df
        self.transform = transform
        self.input_folder= "../input/nfl-impact-detection-train-frames"
            
    def __len__(self) :
        return len(self.df)
    
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x,y,w,h = row[['x','y','w','h']]
        x = int(x)
        y = int(y)
        w = int(w)
        h = int(h)
#         buffer_folder = row.image_id.replace('.png','')
        buffer_folder = '_'.join(row.image_id.split('_')[:3])
        img_path = os.path.join(self.input_folder, buffer_folder, row.image_id)
        
#         print(img_path)
        
        image = cv2.imread(img_path)
#         print(image)
        h = 720
        w =1280
        y2 = y+h if y+h <1280 else h
        x2 = x+w if x+w <720 else w
#         print(x2,y2)
        image = image[y:y2,x:x2]
        
        image = cv2.resize(image, (100,100))   
    
        if self.transform:
            transformed_img = self.transform(image=image)['image']
#         print(transformed_img)
#         copy_row = row.copy()
        return transformed_img, row.to_dict()

transform = A.Compose(
    [
        Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        ),
        ToTensor()
    ]
)

test_dataset = ImageDatasetBoxes(df,transform= transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [193]:
model = torch.load('../input/nfl-csv/se_resnext50_32x4d_epoch_no_mask3.pt')['model']
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [242]:
model.eval()
with torch.no_grad():            
#     try:
    rows = []
    preds = []
    for batch, rows in tqdm(test_loader) :
        batch = batch.to(device)
        op_rows = rows.copy()
        
        out = model(batch)
        
        out_ = out.argmax(1)
        preds.append(out_.to('cpu').numpy())


100%|██████████| 317/317 [09:15<00:00,  1.75s/it]


In [245]:
preds_op = np.concatenate(preds)

In [247]:
pd.DataFrame(preds_op)[0].value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [209]:
!ls ../input/utils-to-prepare-different-type-of-dataset

__notebook__.ipynb  big_helmet_test.csv   helmet_crop_train
__output__.json     big_helmet_train.csv  well_test_dataset.csv
__results__.html    custom.css		  well_train_dataset.csv
__results___files   helmet_crop_test


In [207]:
pd.DataFrame(np.concatenate(preds))[0].value_counts()

0    18470
1       26
Name: 0, dtype: int64

In [201]:
preds

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
!ls 

In [168]:
preds

[]

In [165]:
def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            iou_score = iou(box1[1:], box2[1:])

            if iou_score < 0.35:
                continue
            else:
                cost_matix[i,j]=0

    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

,image_id,category_id,score,x,y,w,h
0,57583_000082_Endzone_034.png,0,0.999986,721.682190,219.008087,21.668213,27.973236
1,57583_000082_Endzone_034.png,0,0.999984,963.926819,245.999466,20.739258,28.251358
2,57583_000082_Endzone_034.png,0,0.999980,556.494812,462.678345,21.307373,20.062500
3,57583_000082_Endzone_034.png,0,0.999977,1207.106201,168.313507,22.421387,27.463409
4,57583_000082_Endzone_034.png,0,0.999975,947.314392,189.925537,20.587646,24.750397
...,...,...,...,...,...,...,...
37856,58104_000352_Sideline_187.png,0,0.996708,532.943542,416.640503,12.297852,17.301636
37857,58104_000352_Sideline_187.png,0,0.993401,544.001160,481.328247,12.367310,14.757874
37858,58104_000352_Sideline_187.png,0,0.992077,746.424927,445.458679,12.172363,14.041931
37859,58104_000352_Sideline_187.png,0,0.982033,701.351685,440.789825,12.585571,16.168030


In [ ]:
np.array([1,1]).tolist()

In [ ]:
# im = cv2.imread('../input/nfl-impact-detection-train-frames/57583_000082_Endzone/57583_000082_Endzone_001.png')
# height, width = im.shape[:2]
# image = torch.as_tensor(im.astype("float32").transpose(2, 0, 1))
# inputs = [{"image": image, "height": height, "width": width }]
# # 'image_sizes':(480,640)}

# model = build_model(cfg)

# # trainer = NflImpactTrainer(cfg)
# DetectionCheckpointer(model).load('../input/nflimpactdetectron/helmet_detector_model_0093999.pth')


# model.eval()#make sure its in eval mode
# #run model
# with torch.no_grad():
#     images = model.preprocess_image(inputs) 
#     features = model.backbone(images.tensor.to('cuda'))
#     proposals, _ = model.proposal_generator(images, features)
# #     outter_proposals = transform(proposals)
#     instances, _ = model.roi_heads(images, features, proposals)
